Import your data here.  I am importing a Pandas DataFrame from a module that queries a postgresql database.  

In [19]:
import warnings
#warnings.filterwarnings('ignore')

import pandas as pd
import datetime as dt

from lifelines import KaplanMeierFitter, CoxPHFitter


import plotly.graph_objs as go
import plotly
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected = True)
import plotly.offline as offline

In [3]:
from data.gen_data import survivorData
survivorData.head()

,prospect,role,segment,account_type,create_date,min_date,max_date
0,AimHo@example115108.com,Other,Real Estate,Prospect,2018-10-24,2018-10-24,2018-11-02
1,AimKim@example76110.com,Decision Maker,Real Estate,Prospect,2018-06-20,NaT,NaT
2,AimLee@example3368.com,Decision Maker,Real Estate,Prospect,2017-09-23,NaT,NaT
3,Aimnez@example44255.com,Decision Maker,Other,Prospect,2018-02-23,2018-02-23,2018-02-23
4,Aimrry@example103606.com,Decision Maker,Real Estate,Prospect,2017-12-17,NaT,NaT


When conducting survival analysis as part of a medical trial, 'deaths' are easily observed.  Likewise, customers signal a 'death' by cancelling a contract, etc.  Prospects, on the other hand, rarely provide a clear signal that they should no longer be considered viable.  So, it is up to the analyst to define 'death' in their prospect base.  

In this example, we will define a prospect death as a specific length of time without engagement from that prospect.  This could be either because the prospect never engaged with us or because, after some interval where they did engage with us, they ceased.  

Of course, that leaves us with the task of determining what that interval of time should be.  In other words, how long can a prospect exist before I can declare him dead OR how long do I wait after a prospect has been active before I can declare him dead?  

To answer the first question, let's find the time that the 80<sup>th</sup> percentile of prospects are in the system before they engage with us.  Using this criteria, the number of days we should allow prospects <b> who have not yet engaged with us</b> to be in the system is:

In [4]:
survivorCreate = survivorData.dropna()
survivorCreate['t_create'] = (survivorCreate.min_date - survivorCreate.create_date).dt.days
maxCreate = int(survivorCreate.t_create.quantile(.80)) 
maxCreate

138

To answer the second question, lets take the time that 80<sup>th</sup> percentile of the prospects who have engaged take between their first and last engagement.  So for those prospects <b> who have engaged</b> with us, the number of days we should wait after their last engagement before declaring them 'dead' is:

In [5]:
survivorEngage = survivorData.dropna()
survivorEngage['t_engage'] = (survivorEngage.max_date - survivorEngage.min_date).dt.days
maxEngage = int(survivorEngage.t_engage.quantile(.80)) 
maxEngage

107

So, now that we have defined "death", we just need to check each record to see if that prospect is alive or not:

In [6]:
todayDate = dt.date(2019,4,30)

# Create function that indicates an observed "death".

def observed(max_date,create_date):
    if pd.notna(max_date):
        if (todayDate - max_date).days > maxEngage:
            return 1 
        else: return 0
    elif (todayDate - create_date).days > maxCreate:
        return 1
    else:
        return 0
        

# Apply function to dataframe       
survivorData['observed'] = survivorData.apply(lambda x: observed(x['max_date'],x['create_date']),axis=1)

And then, we need to calculate the duration of engagement for each prospect:

In [7]:
# Create function that calculates duration.
def duration(max_date, create_date,observed):
    if pd.isnull(max_date):
        if observed == 1:  #If no engagement occurred and we can conclude that the prospect is inactive, the duration is 0
            return 0
        else:
            return (todayDate - create_date).days  #If no engagement occurred but not enough time has passed to rule them inactive....
    else:
        if observed == 1:
            return (max_date - create_date).days
        else:
            return (todayDate - create_date).days

# Apply function to dataframe
survivorData['duration'] = survivorData.apply(lambda x: duration(x['max_date'],x['create_date'],x['observed']),axis=1)

Our final Dataframe looks like this:

In [8]:
survivorData.head()

,prospect,role,segment,account_type,create_date,min_date,max_date,observed,duration
0,AimHo@example115108.com,Other,Real Estate,Prospect,2018-10-24,2018-10-24,2018-11-02,1,9
1,AimKim@example76110.com,Decision Maker,Real Estate,Prospect,2018-06-20,NaT,NaT,1,0
2,AimLee@example3368.com,Decision Maker,Real Estate,Prospect,2017-09-23,NaT,NaT,1,0
3,Aimnez@example44255.com,Decision Maker,Other,Prospect,2018-02-23,2018-02-23,2018-02-23,1,0
4,Aimrry@example103606.com,Decision Maker,Real Estate,Prospect,2017-12-17,NaT,NaT,1,0


Now that our data has been wrangled, and the observations and durations have been calculated, we can fit our survival curve using the Kaplan Meir model provided in the [lifelines](https://lifelines.readthedocs.io/en/latest/) package: 

In [9]:
T=survivorData['duration']
E=survivorData['observed']

kmf = KaplanMeierFitter()
survTable = kmf.fit(T,event_observed=E).survival_function_.reset_index()

And then plot using plotly:

In [20]:
trace = go.Scatter(x=survTable['timeline'],y=survTable['KM_estimate'])
layout = go.Layout(yaxis={'range':[0,1],'showline':False,'title':'Survival Probability'},xaxis={'title':'Duration (days)'})
fig=go.Figure(data=[trace],layout=layout)
offline.iplot(fig)

Virtually any other survival curve will start with 100% survival rate because in clinical trials, etc. patients who are already deceased will not be enrolling in a study.  The same thing could be said for a survival curve of customers.  However, since we are dealing with Prospects, I think it is important to show how many are essentially non-responsive from the beginning.  In the case of this client, only around 13% of the selected prospects ever engaged with any marketing effort.  Or alternatively, 87% of prospects were dead on arrival.  (Remember that this group of prospects should be valid - i.e. they have an active company email, are in the targeted geography, etc.)

The other striking element of the above chart is the duration of engagement.  It looks as if, once engaged, prospects tend to stay engaged for a long period of time.  However, remember that, in this group, we included clients.  So, let's look at the chart that includes only those records that have not yet converted.  

In [17]:
survProsp = survivorData[survivorData.account_type == 'Prospect']

T=survProsp['duration']
E=survProsp['observed']

kmf = KaplanMeierFitter()
survProspTable = kmf.fit(T,event_observed=E).survival_function_.reset_index()

survClient = survivorData[survivorData.account_type == 'Client']

T=survClient['duration']
E=survClient['observed']

kmf = KaplanMeierFitter()
survClientTable = kmf.fit(T,event_observed=E).survival_function_.reset_index()

traceProsp = go.Scatter(x=survProspTable['timeline'],y=survProspTable['KM_estimate'])
traceClient = go.Scatter(x=survClientTable['timeline'],y=survClientTable['KM_estimate'])

layout = go.Layout(yaxis={'range':[0,1],'showline':False,'title':'Survival Probability'},xaxis={'title':'Duration (days)'})
fig=go.Figure(data=[traceProsp,traceClient],layout=layout)
offline.iplot(fig)

So, we can see from these curves that, as we would expect, those prospects who go on to become clients engage at a much higher rate (30%) and remain engaged at a much higher rate than those who never convert.  While a 30% engagement rate may still seem low, you have to remember that group is made up all roles (decision makers may engage at a higher rate for example), all segments, etc.  We could explore each of these parameters by producing charts and attempting to compare them with each other but that would be labor intensive and probably somewhat confusing.  Instead let's perform a survival regression using the Cox Proportional Hazards model: 

In [13]:
regData = survivorData[['role','account_type','segment','duration','observed']]
regData = pd.get_dummies(regData)  # We have to replace our categorical variables (segment, etc.) with dummy variables.
regData.drop(['role_Other','account_type_Client','segment_Other'],axis=1,inplace=True) # Drop one column from each dummy to prevent high-colinearity issue.
cph = CoxPHFitter()
cph.fit(regData,'duration',event_col='observed')
cph.print_summary()

<lifelines.CoxPHFitter: fitted with 225016 observations, 15111 censored>
      duration col = 'duration'
         event col = 'observed'
number of subjects = 225016
  number of events = 209905
    log-likelihood = -2400775.75
  time fit was run = 2019-05-03 14:21:37 UTC

---
                       coef exp(coef)  se(coef)      z      p  -log2(p)  lower 0.95  upper 0.95
role_Decision Maker    0.15      1.17      0.01  27.31 <0.005    542.95        0.14        0.17
role_Influencer        0.12      1.13      0.01  18.46 <0.005    250.25        0.11        0.13
account_type_Prospect  0.45      1.57      0.01  63.09 <0.005       inf        0.44        0.46
segment_Banking       -0.08      0.92      0.01 -14.60 <0.005    157.97       -0.09       -0.07
segment_Real Estate    0.06      1.06      0.01  10.47 <0.005     82.87        0.05        0.07
---
Concordance = 0.68
Log-likelihood ratio test = 10381.55 on 5 df, -log2(p)=inf
